In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Input, Embedding, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
import os
import tensorflow as tf 
from tensorflow.python.client import device_lib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(device_lib.list_local_devices() )

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16179639674331976379
xla_global_id: -1
]


In [54]:
df = pd.read_csv("./data/pm10_final_minor.csv", index_col = 0)

In [29]:
df.head()

,"('PM10', '강원 강릉시')","('PM10', '광주 북구')","('PM10', '대전 중구')","('PM10', '부산 해운대구')","('PM10', '서울 서대문구')","('풍속', '21229')","('풍속', '22101')","('풍속', '22102')","('풍속', '22103')","('풍속', '22104')",...,month_11,month_12,hour_0,hour_3,hour_6,hour_9,hour_12,hour_15,hour_18,hour_21
2015-01-01 03:00:00,35.0,21.7,33.3,18.7,63.7,11.0,12.0,14.0,13.0,10.0,...,False,False,False,True,False,False,False,False,False,False
2015-01-01 06:00:00,45.3,21.0,61.3,19.0,113.3,14.0,12.0,13.0,14.0,11.0,...,False,False,False,False,True,False,False,False,False,False
2015-01-01 09:00:00,56.3,21.7,95.0,28.3,135.0,12.0,12.0,15.0,14.0,12.0,...,False,False,False,False,False,True,False,False,False,False
2015-01-01 12:00:00,68.7,41.0,124.0,53.5,85.0,12.0,10.0,13.0,14.0,12.0,...,False,False,False,False,False,False,True,False,False,False
2015-01-01 15:00:00,88.3,70.3,56.3,54.3,44.7,13.0,10.0,13.0,14.0,8.0,...,False,False,False,False,False,False,False,True,False,False


In [30]:
df.value_counts

<bound method DataFrame.value_counts of                      ('PM10', '강원 강릉시')  ('PM10', '광주 북구')  ('PM10', '대전 중구')  \
2015-01-01 03:00:00                35.0               21.7               33.3   
2015-01-01 06:00:00                45.3               21.0               61.3   
2015-01-01 09:00:00                56.3               21.7               95.0   
2015-01-01 12:00:00                68.7               41.0              124.0   
2015-01-01 15:00:00                88.3               70.3               56.3   
...                                 ...                ...                ...   
2024-05-19 09:00:00                22.0               24.7               17.3   
2024-05-19 12:00:00                25.0               21.3               30.0   
2024-05-19 15:00:00                45.3               14.0               19.0   
2024-05-19 18:00:00                34.3               20.7               16.3   
2024-05-19 21:00:00                35.7               20.0           

In [31]:
df.columns[:5]

Index(['('PM10', '강원 강릉시')', '('PM10', '광주 북구')', '('PM10', '대전 중구')',
       '('PM10', '부산 해운대구')', '('PM10', '서울 서대문구')'],
      dtype='object')

In [55]:
df.drop(df.loc[:, ["('PM10', '강원 강릉시')", "('PM10', '광주 북구')", "('PM10', '대전 중구')", "('PM10', '부산 해운대구')"]].columns, axis = 1)

,"('PM10', '서울 서대문구')","('풍속', '21229')","('풍속', '22101')","('풍속', '22102')","('풍속', '22103')","('풍속', '22104')","('풍속', '22105')","('풍속', '22106')","('풍속', '22107')","('풍속', '22108')",...,month_11,month_12,hour_0,hour_3,hour_6,hour_9,hour_12,hour_15,hour_18,hour_21
2015-01-01 03:00:00,63.7,11.0,12.0,14.0,13.0,10.0,13.0,12.5,14.0,13.0,...,False,False,False,True,False,False,False,False,False,False
2015-01-01 06:00:00,113.3,14.0,12.0,13.0,14.0,11.0,13.0,13.1,15.0,14.0,...,False,False,False,False,True,False,False,False,False,False
2015-01-01 09:00:00,135.0,12.0,12.0,15.0,14.0,12.0,13.0,12.6,10.0,14.0,...,False,False,False,False,False,True,False,False,False,False
2015-01-01 12:00:00,85.0,12.0,10.0,13.0,14.0,12.0,13.0,12.3,14.0,13.0,...,False,False,False,False,False,False,True,False,False,False
2015-01-01 15:00:00,44.7,13.0,10.0,13.0,14.0,8.0,11.0,11.6,13.0,12.0,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-19 09:00:00,18.0,2.5,1.0,2.0,2.0,1.0,2.0,5.0,4.0,2.0,...,False,False,False,False,False,True,False,False,False,False
2024-05-19 12:00:00,17.0,3.0,0.0,2.0,2.0,1.0,2.0,2.0,3.0,2.0,...,False,False,False,False,False,False,True,False,False,False
2024-05-19 15:00:00,18.3,5.0,3.3,3.3,1.0,4.0,7.0,5.0,2.0,2.0,...,False,False,False,False,False,False,False,True,False,False
2024-05-19 18:00:00,24.7,3.0,1.0,2.0,3.0,5.0,7.0,2.0,2.0,1.0,...,False,False,False,False,False,False,False,False,True,False


In [56]:
df1 = df.drop(df.loc[:, ["('PM10', '강원 강릉시')", "('PM10', '광주 북구')", "('PM10', '대전 중구')", "('PM10', '부산 해운대구')"]].columns, axis = 1)

In [34]:
df1.iloc[:, :5].info()

<class 'pandas.core.frame.DataFrame'>
Index: 27415 entries, 2015-01-01 03:00:00 to 2024-05-19 21:00:00
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ('PM10', '서울 서대문구')  26805 non-null  float64
 1   ('풍속', '21229')      27204 non-null  float64
 2   ('풍속', '22101')      27250 non-null  float64
 3   ('풍속', '22102')      27253 non-null  float64
 4   ('풍속', '22103')      27318 non-null  float64
dtypes: float64(5)
memory usage: 1.3+ MB


In [35]:
df1.shape

(27415, 1871)

In [36]:
df1.head()

,"('PM10', '서울 서대문구')","('풍속', '21229')","('풍속', '22101')","('풍속', '22102')","('풍속', '22103')","('풍속', '22104')","('풍속', '22105')","('풍속', '22106')","('풍속', '22107')","('풍속', '22108')",...,month_11,month_12,hour_0,hour_3,hour_6,hour_9,hour_12,hour_15,hour_18,hour_21
2015-01-01 03:00:00,63.7,11.0,12.0,14.0,13.0,10.0,13.0,12.5,14.0,13.0,...,False,False,False,True,False,False,False,False,False,False
2015-01-01 06:00:00,113.3,14.0,12.0,13.0,14.0,11.0,13.0,13.1,15.0,14.0,...,False,False,False,False,True,False,False,False,False,False
2015-01-01 09:00:00,135.0,12.0,12.0,15.0,14.0,12.0,13.0,12.6,10.0,14.0,...,False,False,False,False,False,True,False,False,False,False
2015-01-01 12:00:00,85.0,12.0,10.0,13.0,14.0,12.0,13.0,12.3,14.0,13.0,...,False,False,False,False,False,False,True,False,False,False
2015-01-01 15:00:00,44.7,13.0,10.0,13.0,14.0,8.0,11.0,11.6,13.0,12.0,...,False,False,False,False,False,False,False,True,False,False


In [57]:
x = df1.drop("('PM10', '서울 서대문구')", axis = 1)
y = df1["('PM10', '서울 서대문구')"]

In [58]:
# 시계열 예측과 같은 작업에서는 단일 스택을 사용하여 입력 시퀀스를 처리
def create_seq2seq_data(X, y, input_steps=48, output_steps=24): # input_steps는 우리가 예측하는 시간이고 output_steps는 예측할 시간(output_steps는 고정)
    Xs, ys = [], []
    for i in range(len(X) - input_steps - output_steps):
        print(i,'/',len(X)-input_steps-output_steps)
        xs_seq = X.iloc[i:(i + input_steps)].values
        ys_seq = y.iloc[(i + input_steps):(i + input_steps + output_steps)].values
        if (not any(np.isnan(i) for x in xs_seq for i in x)) & (not any(np.isnan(y) for y in ys_seq)):
            Xs.append(xs_seq)
            ys.append(ys_seq)
    return np.array(Xs).astype(np.float32), np.array(ys).astype(np.float32)

In [60]:
input_steps = 48
output_steps = 24
X_seq2seq, y_seq2seq = create_seq2seq_data(x, y, input_steps, output_steps)

0 / 27343
1 / 27343
2 / 27343
3 / 27343
4 / 27343
5 / 27343
6 / 27343
7 / 27343
8 / 27343
9 / 27343
10 / 27343
11 / 27343
12 / 27343
13 / 27343
14 / 27343
15 / 27343
16 / 27343
17 / 27343
18 / 27343
19 / 27343
20 / 27343
21 / 27343
22 / 27343
23 / 27343
24 / 27343
25 / 27343
26 / 27343
27 / 27343
28 / 27343
29 / 27343
30 / 27343
31 / 27343
32 / 27343
33 / 27343
34 / 27343
35 / 27343
36 / 27343
37 / 27343
38 / 27343
39 / 27343
40 / 27343
41 / 27343
42 / 27343
43 / 27343
44 / 27343
45 / 27343
46 / 27343
47 / 27343
48 / 27343
49 / 27343
50 / 27343
51 / 27343
52 / 27343
53 / 27343
54 / 27343
55 / 27343
56 / 27343
57 / 27343
58 / 27343
59 / 27343
60 / 27343
61 / 27343
62 / 27343
63 / 27343
64 / 27343
65 / 27343
66 / 27343
67 / 27343
68 / 27343
69 / 27343
70 / 27343
71 / 27343
72 / 27343
73 / 27343
74 / 27343
75 / 27343
76 / 27343
77 / 27343
78 / 27343
79 / 27343
80 / 27343
81 / 27343
82 / 27343
83 / 27343
84 / 27343
85 / 27343
86 / 27343
87 / 27343
88 / 27343
89 / 27343
90 / 27343
91 / 2734

In [61]:
x_train, x_test, y_train, y_test = train_test_split(X_seq2seq, y_seq2seq, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [62]:
ss = StandardScaler()

In [63]:
train_reshaped = x_train.reshape(x_train.shape[0], -1)
val_reshaped = x_val.reshape(x_val.shape[0], -1)
test_reshaped = x_test.reshape(x_test.shape[0], -1)

train_scaled_reshaped = ss.fit_transform(train_reshaped)
val_scaled_reshaped = ss.transform(val_reshaped)
test_scaled_reshaped = ss.transform(test_reshaped)

scaled_train = train_scaled_reshaped.reshape(x_train.shape)
scaled_val = val_scaled_reshaped.reshape(x_val.shape)
scaled_test = test_scaled_reshaped.reshape(x_test.shape)

## Multi-Head-Attention

<img src="./image/multi-head-attention.png">

In [64]:
# 트랜스포머 모델 정의 - 트랜스포머 모델의 입력 데이터 형식은 (batch_size, sequence_length, num_features)
class MultiHeadSelfAttention(Layer): # 입력으로 주어진 시퀀스를 각 헤드에 대해 독립적으로 어텐션을 수행하고, 그 결과를 결합하여 출력(병렬구조)
    def __init__(self, embed_dim, num_heads=8): # embed_dim은 입력 임베딩 차원, num_heads는 어텐션을 수행할 헤드의 수
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0: # 입력 임베딩 차원이 헤드의 수로 나누어 떨어지지 않으면 ValueError가 발생
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads # 각 헤드의 프로젝션 차원은 embed_dim // num_heads
        # query_dense, key_dense, value_dense 및 combine_heads는 각각 어텐션에 사용되는 dense 레이어
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)

    # 정보 손실 문제 해결
    def attention(self, query, key, value): # attention 함수는 주어진 쿼리, 키 및 값에 대해 어텐션 가중치를 계산하고, 이를 사용하여 값을 가중합하여 출력을 생성
        score = tf.matmul(query, key, transpose_b=True) # matmul()는 행렬 곱셈을 수행할 때 사용되는 함수이며, 두 번째 행렬 b를 전치(transpose)하여 곱하는 것을 의미
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size): # separate_heads 함수는 입력을 여러 헤드로 분리하는 메서드
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim)) # 입력 형태는 (batch_size, num_heads, seq_len, projection_dim) 형태로 변환
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs): # 입력에 대한 어텐션을 계산하는 메서드
        batch_size = tf.shape(inputs)[0]
        # 입력에 대해 query_dense, key_dense, value_dense 레이어를 통과시키고, 그 결과를 각각 separate_heads() 메서드로 분리
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value) # 분리된 쿼리, 키 및 값에 대해 어텐션을 계산하고, 이를 결합
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim)) 
        output = self.combine_heads(concat_attention) # 마지막으로, 결합된 어텐션 결과에 combine_heads 레이어를 적용하여 최종 출력을 생성
        return output

## FFN & Layer_normalization

<img src="./image/FFN.png">
<img src="./image/FFN2.png">
<img src="./image/layer_normalization.png">
<img src="./image/layer_normalization_regularization.png">

In [65]:
class TransformerBlock(Layer): # TransformerBlock은 여러 층으로 쌓여 전체 트랜스포머 모델을 형성하며, 입력 시퀀스의 특징을 추출하고 다음 층으로 전달
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1): # embed_dim은 입력 임베딩 차원, num_heads는 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수
        # ff_dim은 피드 포워드 신경망의 은닉층 크기, rate는 dropout의 비율
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads) # att는 멀티 헤드 셀프 어텐션 레이어를 생성
        self.ffn = Sequential( # ffn은 피드 포워드 신경망을 생성. 이는 두 개의 dense 레이어로 구성되어 있음. 
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),] # 첫 번째 레이어는 ReLU 활성화 함수를 사용하고, 두 번째 레이어는 선형 활성화 함수를 사용
        )
        # layernorm1, layernorm2는 레이어 정규화 레이어
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        # dropout1, 2 레이어도 설정
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False): # 입력을 받아와서 먼저 멀티 헤드 셀프 어텐션을 수행
        # 어텐션 출력에 드롭아웃을 적용하고, 입력과 어텐션 출력을 더한 후 레이어 정규화를 수행
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        # 그 다음, 피드 포워드 신경망을 통해 처리된 출력에 드롭아웃을 적용
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        # 마지막으로, 이전 출력과 더한 후 다시 레이어 정규화를 수행하여 최종 출력을 생성
        return self.layernorm2(out1 + ffn_output)

## Positional encoding

<img src="./image/positional_encoding.png">
<img src="./image/positional_embedding_example.png">

In [66]:
class TokenAndPositionEmbedding(Layer): # 트랜스포머 모델의 입력으로 사용될 토큰 및 위치 임베딩을 생성하는 레이어를 정의
                                        # 입력 시퀀스의 각 토큰에 대한 임베딩과 해당 위치에 대한 임베딩을 결합하여 최종 임베딩을 생성
    def __init__(self, maxlen, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.maxlen = maxlen # maxlen은 입력 시퀀스의 최대 길이
        self.embed_dim = embed_dim # embed_dim은 토큰 및 위치 임베딩의 차원
        self.token_emb = Dense(embed_dim) # token_emb는 입력 토큰에 대한 임베딩을 생성하는 밀집 레이어
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim) # pos_emb는 입력 시퀀스의 각 위치에 대한 임베딩을 생성하는 임베딩 레이어
                                                                         # 위치 임베딩의 크기는 (maxlen, embed_dim)

    def call(self, x): # 입력으로 주어진 토큰 시퀀스에 대해 위치 임베딩을 생성
        maxlen = tf.shape(x)[-2] 
        positions = tf.range(start=0, limit=maxlen, delta=1)
        # 각 위치에 대한 임베딩은 Embedding 레이어를 사용하여 생성되며, 시퀀스의 각 위치에 해당하는 임베딩을 가져와서 positions 변수에 저장
        positions = self.pos_emb(positions)
        token_embeddings = self.token_emb(x) # 입력 토큰에 대한 임베딩은 token_emb 레이어를 사용하여 생성
        return token_embeddings + positions # 위치 임베딩과 토큰 임베딩을 더하여 최종 임베딩을 생성하고 반환

# 문장을 한번에 병렬 처리해버리는 Transformer에 단어 순서를 알려주기 위한 작업
# 이 모델을 사용하면 입력 시퀀스의 각 토큰에 대해 임베딩을 생성하고, 해당 토큰의 위치 정보를 포함하여 트랜스포머 모델에 입력으로 전달

## Transformer 구조

<img src="./image/transformer.png">

In [67]:
def build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks): # 트랜스포머 모델을 구축하는 데 사용
    # 모델의 입력 형태, 출력의 스텝 수, 임베딩 차원, 어텐션 헤드 수, 피드 포워드 신경망의 은닉층 크기, 트랜스포머 블록의 수
    inputs = Input(shape=input_shape) # 입력 데이터를 받기 위한 Keras의 Input 레이어를 생성
    embedding_layer = TokenAndPositionEmbedding(input_shape[0], embed_dim) # TokenAndPositionEmbedding 클래스를 사용하여 입력 데이터에 토큰 및 위치 임베딩을 적용
    x = embedding_layer(inputs)
    for _ in range(num_transformer_blocks): # 지정된 개수의 트랜스포머 블록을 생성하고 이전 블록의 출력을 현재 블록의 입력으로 전달
        x = TransformerBlock(embed_dim, num_heads, ff_dim)(x) # 모든 트랜스포머 블록이 생성된 후, 출력을 하나의 값으로 예측하기 위해 Dense 레이어를 적용
    x = Dense(1)(x)  # 수정된 부분: Dense(1)을 사용하여 각 타임 스텝에 대해 하나의 값을 예측
    x = tf.keras.layers.Flatten()(x)  # 수정된 부분: 출력을 평탄화하여 맞는 차원으로 변환
    outputs = Dense(output_steps)(x)  # 최종 출력 레이어
    model = Model(inputs=inputs, outputs=outputs) # 출력을 평탄화하여 차원을 맞추고, 최종 출력 레이어를 추가하여 최종 모델을 생성
    return model

In [ ]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 트랜스포머블록의 수, 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm10_checkpoint\pm10_ver1\transformer-model.h5", save_best_only=True) 
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_3 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_6 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 transformer_block_7 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 dense_58 (Dense)            (None, 48, 1)             33  

In [31]:
# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Epoch 1/200
426/426 [==============================] - 7s 12ms/step - loss: 668.0885 - val_loss: 458.4188
Epoch 2/200
426/426 [==============================] - 5s 11ms/step - loss: 426.0230 - val_loss: 416.0105
Epoch 3/200
426/426 [==============================] - 4s 11ms/step - loss: 391.5809 - val_loss: 395.2868
Epoch 4/200
426/426 [==============================] - 4s 11ms/step - loss: 382.3247 - val_loss: 402.8743
Epoch 5/200
426/426 [==============================] - 5s 11ms/step - loss: 392.3031 - val_loss: 389.3559
Epoch 6/200
426/426 [==============================] - 4s 10ms/step - loss: 379.6615 - val_loss: 388.3806
Epoch 7/200
426/426 [==============================] - 4s 11ms/step - loss: 376.7852 - val_loss: 379.2710
Epoch 8/200
426/426 [==============================] - 4s 10ms/step - loss: 377.7036 - val_loss: 384.1346
Epoch 9/200
426/426 [==============================] - 4s 10ms/step - loss: 378.7815 - val_loss: 389.5169
Epoch 10/200
426/426 [========================

In [ ]:
predictions[:10]

In [ ]:
y_test[:10]

In [ ]:
predictions.shape

In [25]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm10_checkpoint\pm10_ver1\transformer_minor0-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 406)]         0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            14560     
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  (None, 48, 32)            6464      
 rmerBlock)                                                      
                                                                 
 transformer_block_1 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 dense_13 (Dense)            (None, 48, 1)             33    

c:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


431/431 [==============================] - 3s 7ms/step - loss: 749.5910 - val_loss: 786.1542
Epoch 3/200
431/431 [==============================] - 3s 7ms/step - loss: 694.7538 - val_loss: 768.1478
Epoch 4/200
431/431 [==============================] - 3s 7ms/step - loss: 671.0972 - val_loss: 702.2238
Epoch 5/200
431/431 [==============================] - 3s 7ms/step - loss: 641.5348 - val_loss: 700.1465
Epoch 6/200
431/431 [==============================] - 3s 7ms/step - loss: 624.3341 - val_loss: 676.4843
Epoch 7/200
431/431 [==============================] - 3s 7ms/step - loss: 621.4116 - val_loss: 666.2302
Epoch 8/200
431/431 [==============================] - 3s 7ms/step - loss: 605.4306 - val_loss: 653.0714
Epoch 9/200
431/431 [==============================] - 3s 7ms/step - loss: 601.4027 - val_loss: 673.4313
Epoch 10/200
431/431 [==============================] - 3s 7ms/step - loss: 586.2654 - val_loss: 623.3856
Epoch 11/200
431/431 [==============================] - 3s 7ms/ste

In [26]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm10_checkpoint\pm10_ver1\transformer_minor0(1)-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 406)]         0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            14560     
 ng_1 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_2 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 transformer_block_3 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 dense_28 (Dense)            (None, 48, 1)             33  

In [68]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm10_checkpoint\pm10_ver1\transformer_minor0(2)-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 48, 406)]         0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            14560     
 ng_8 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_16 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_17 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_133 (Dense)           (None, 48, 1)             33  

c:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


431/431 [==============================] - 5s 11ms/step - loss: 771.9444 - val_loss: 822.1536
Epoch 3/200
431/431 [==============================] - 6s 13ms/step - loss: 709.8475 - val_loss: 740.3770
Epoch 4/200
431/431 [==============================] - 5s 11ms/step - loss: 674.1241 - val_loss: 763.6806
Epoch 5/200
431/431 [==============================] - 6s 13ms/step - loss: 651.4383 - val_loss: 692.4392
Epoch 6/200
431/431 [==============================] - 5s 11ms/step - loss: 636.7257 - val_loss: 688.3064
Epoch 7/200
431/431 [==============================] - 6s 13ms/step - loss: 626.1229 - val_loss: 687.9163
Epoch 8/200
431/431 [==============================] - 5s 10ms/step - loss: 653.2871 - val_loss: 686.0515
Epoch 9/200
431/431 [==============================] - 5s 12ms/step - loss: 619.9725 - val_loss: 673.7591
Epoch 10/200
431/431 [==============================] - 5s 11ms/step - loss: 649.0438 - val_loss: 763.4935
Epoch 11/200
431/431 [==============================] - 4

In [47]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm10_checkpoint\pm10_ver1\transformer_minor6-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 48, 1870)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            61408     
 ng_3 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_6 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 transformer_block_7 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 dense_58 (Dense)            (None, 48, 1)             33  

c:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


431/431 [==============================] - 5s 11ms/step - loss: 684.6230 - val_loss: 712.8682
Epoch 3/200
431/431 [==============================] - 5s 11ms/step - loss: 630.8695 - val_loss: 687.7634
Epoch 4/200
431/431 [==============================] - 5s 11ms/step - loss: 637.6164 - val_loss: 684.8851
Epoch 5/200
431/431 [==============================] - 5s 11ms/step - loss: 618.9490 - val_loss: 669.9731
Epoch 6/200
431/431 [==============================] - 5s 11ms/step - loss: 610.5698 - val_loss: 665.3690
Epoch 7/200
431/431 [==============================] - 5s 11ms/step - loss: 599.1177 - val_loss: 666.6498
Epoch 8/200
431/431 [==============================] - 5s 11ms/step - loss: 603.2492 - val_loss: 651.0312
Epoch 9/200
431/431 [==============================] - 5s 11ms/step - loss: 607.2161 - val_loss: 696.6630
Epoch 10/200
431/431 [==============================] - 5s 11ms/step - loss: 606.9658 - val_loss: 652.0699
Epoch 11/200
431/431 [==============================] - 5

In [48]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm10_checkpoint\pm10_ver1\transformer_minor6(1)-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 48, 1870)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            61408     
 ng_4 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_8 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 transformer_block_9 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 dense_73 (Dense)            (None, 48, 1)             33  

In [49]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm10_checkpoint\pm10_ver1\transformer_minor6(2)-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 48, 1870)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            61408     
 ng_5 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_10 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_11 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_88 (Dense)            (None, 48, 1)             33  

In [113]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_kangwon1-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_18 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_36 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_37 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_283 (Dense)           (None, 48, 1)             33 

c:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


222/222 [==============================] - 6s 28ms/step - loss: 62.4552 - val_loss: 60.4980
Epoch 3/200
222/222 [==============================] - 6s 27ms/step - loss: 58.8450 - val_loss: 58.8525
Epoch 4/200
222/222 [==============================] - 6s 27ms/step - loss: 57.1578 - val_loss: 57.9308
Epoch 5/200
222/222 [==============================] - 6s 27ms/step - loss: 53.4379 - val_loss: 54.2311
Epoch 6/200
222/222 [==============================] - 6s 27ms/step - loss: 47.4188 - val_loss: 46.9466
Epoch 7/200
222/222 [==============================] - 6s 27ms/step - loss: 39.9428 - val_loss: 37.1544
Epoch 8/200
222/222 [==============================] - 6s 27ms/step - loss: 30.7092 - val_loss: 30.3155
Epoch 9/200
222/222 [==============================] - 6s 27ms/step - loss: 24.5562 - val_loss: 25.8683
Epoch 10/200
222/222 [==============================] - 6s 27ms/step - loss: 20.7704 - val_loss: 21.3373
Epoch 11/200
222/222 [==============================] - 6s 27ms/step - loss

In [114]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_kangwon2-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_19 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_38 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_39 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_298 (Dense)           (None, 48, 1)             33 

In [115]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_kangwon3-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_20 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_40 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_41 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_313 (Dense)           (None, 48, 1)             33 

In [131]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_kwangju1-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_21 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_42 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_43 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_328 (Dense)           (None, 48, 1)             33 

c:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/200
241/241 [==============================] - 8s 32ms/step - loss: 122.9297 - val_loss: 128.9830
Epoch 3/200
241/241 [==============================] - 8s 32ms/step - loss: 113.1127 - val_loss: 108.9650
Epoch 4/200
241/241 [==============================] - 8s 31ms/step - loss: 109.2479 - val_loss: 107.3104
Epoch 5/200
241/241 [==============================] - 8s 32ms/step - loss: 105.3193 - val_loss: 105.3847
Epoch 6/200
241/241 [==============================] - 7s 30ms/step - loss: 104.0875 - val_loss: 102.1738
Epoch 7/200
241/241 [==============================] - 7s 29ms/step - loss: 100.6487 - val_loss: 98.7716
Epoch 8/200
241/241 [==============================] - 7s 29ms/step - loss: 95.0573 - val_loss: 97.1724
Epoch 9/200
241/241 [==============================] - 7s 29ms/step - loss: 86.2853 - val_loss: 80.0173
Epoch 10/200
241/241 [==============================] - 7s 29ms/step - loss: 71.3008 - val_loss: 67.0343
Epoch 11/200
241/241 [==============================

In [132]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_kwangju2-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_22 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_44 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_45 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_343 (Dense)           (None, 48, 1)             33 

In [133]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_kwangju3-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_23 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_46 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_47 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_358 (Dense)           (None, 48, 1)             33 

In [149]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_daejeon1-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_24 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_48 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_49 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_373 (Dense)           (None, 48, 1)             33 

c:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


249/249 [==============================] - 5s 18ms/step - loss: 100.6512 - val_loss: 95.8083
Epoch 3/200
249/249 [==============================] - 5s 18ms/step - loss: 92.2941 - val_loss: 90.0520
Epoch 4/200
249/249 [==============================] - 5s 18ms/step - loss: 89.3392 - val_loss: 87.9551
Epoch 5/200
249/249 [==============================] - 5s 18ms/step - loss: 86.8565 - val_loss: 85.5685
Epoch 6/200
249/249 [==============================] - 5s 18ms/step - loss: 84.1022 - val_loss: 81.6286
Epoch 7/200
249/249 [==============================] - 5s 18ms/step - loss: 77.9021 - val_loss: 69.0413
Epoch 8/200
249/249 [==============================] - 5s 18ms/step - loss: 63.6096 - val_loss: 56.2575
Epoch 9/200
249/249 [==============================] - 5s 18ms/step - loss: 51.4152 - val_loss: 46.9906
Epoch 10/200
249/249 [==============================] - 5s 18ms/step - loss: 43.1969 - val_loss: 40.2523
Epoch 11/200
249/249 [==============================] - 5s 18ms/step - los

In [150]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_daejeon2-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_25 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_50 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_51 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_388 (Dense)           (None, 48, 1)             33 

In [151]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_daejeon3-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_26 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_52 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_53 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_403 (Dense)           (None, 48, 1)             33 

In [167]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_busan1-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_27 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_54 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_55 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_418 (Dense)           (None, 48, 1)             33 

c:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


260/260 [==============================] - 6s 21ms/step - loss: 67.2658 - val_loss: 65.5504
Epoch 3/200
260/260 [==============================] - 5s 20ms/step - loss: 64.0751 - val_loss: 62.4883
Epoch 4/200
260/260 [==============================] - 5s 20ms/step - loss: 61.4727 - val_loss: 60.9571
Epoch 5/200
260/260 [==============================] - 5s 20ms/step - loss: 57.4317 - val_loss: 56.3948
Epoch 6/200
260/260 [==============================] - 5s 20ms/step - loss: 52.2173 - val_loss: 53.8708
Epoch 7/200
260/260 [==============================] - 5s 20ms/step - loss: 45.5464 - val_loss: 45.0745
Epoch 8/200
260/260 [==============================] - 5s 20ms/step - loss: 40.4415 - val_loss: 38.3074
Epoch 9/200
260/260 [==============================] - 5s 20ms/step - loss: 33.2084 - val_loss: 31.9718
Epoch 10/200
260/260 [==============================] - 5s 20ms/step - loss: 27.6815 - val_loss: 27.6680
Epoch 11/200
260/260 [==============================] - 5s 20ms/step - loss

In [168]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_busan2-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_28 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_56 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_57 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_433 (Dense)           (None, 48, 1)             33 

In [169]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_ver1_busan3-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_29 (TokenAndPositionEmb                                      
 edding)                                                         
                                                                 
 transformer_block_58 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 transformer_block_59 (Tran  (None, 48, 32)            6464      
 sformerBlock)                                                   
                                                                 
 dense_448 (Dense)           (None, 48, 1)             33 

In [23]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_Seoul1-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  (None, 48, 32)            6464      
 rmerBlock)                                                      
                                                                 
 transformer_block_1 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 dense_13 (Dense)            (None, 48, 1)             33    

c:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


254/254 [==============================] - 2s 9ms/step - loss: 165.7433 - val_loss: 164.1619
Epoch 3/200
254/254 [==============================] - 2s 10ms/step - loss: 153.2555 - val_loss: 162.9812
Epoch 4/200
254/254 [==============================] - 2s 10ms/step - loss: 147.3050 - val_loss: 155.6028
Epoch 5/200
254/254 [==============================] - 2s 10ms/step - loss: 144.2038 - val_loss: 149.7576
Epoch 6/200
254/254 [==============================] - 2s 10ms/step - loss: 143.6324 - val_loss: 148.2066
Epoch 7/200
254/254 [==============================] - 3s 10ms/step - loss: 140.1980 - val_loss: 147.8655
Epoch 8/200
254/254 [==============================] - 3s 10ms/step - loss: 137.4226 - val_loss: 142.5800
Epoch 9/200
254/254 [==============================] - 3s 10ms/step - loss: 130.9066 - val_loss: 134.9155
Epoch 10/200
254/254 [==============================] - 3s 10ms/step - loss: 115.2081 - val_loss: 115.9686
Epoch 11/200
254/254 [==============================] - 3s

In [24]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_Seoul2-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_1 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_2 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 transformer_block_3 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 dense_28 (Dense)            (None, 48, 1)             33  

In [25]:
input_shape = (input_steps, x_train.shape[2])
embed_dim = 32 # 입력 임베딩 차원, 값을 늘리면 모델이 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_heads = 2 # 멀티 헤드 셀프 어텐션에서 사용할 어텐션 헤드의 수, 값을 늘리면 모델이 더 다양한 패턴을 학습할 수 있지만, 계산 비용이 증가
ff_dim = 32 # 피드 포워드 신경망의 은닉층 크기, 값을 늘리면 모델의 용량이 증가하여 더 많은 정보를 학습할 수 있지만, 계산 비용이 증가
num_transformer_blocks = 2 # 값을 늘리면 모델이 더 깊어져 더 복잡한 패턴을 학습할 수 있지만, 계산 비용이 증가

model = build_transformer_model(input_shape, output_steps, embed_dim, num_heads, ff_dim, num_transformer_blocks)

model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(r"C:\Users\user\Desktop\koreaIT\pm25_checkpoint\pm25_ver1\transformer_Seoul3-model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
model.summary()

# 모델 학습
history = model.fit(scaled_train, y_train, epochs=200, validation_data=(scaled_val, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# 모델 평가 및 예측
model.evaluate(scaled_test, y_test)
predictions = model.predict(scaled_test)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 48, 2220)]        0         
                                                                 
 token_and_position_embeddi  (None, 48, 32)            72608     
 ng_2 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_4 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 transformer_block_5 (Trans  (None, 48, 32)            6464      
 formerBlock)                                                    
                                                                 
 dense_43 (Dense)            (None, 48, 1)             33  